# Upscaling Service - Proof of Concept
This notebooks showcases a demo of the APEx Upscaling Service by demonstrating the capabilities of the [APEx Dispatcher API](https://github.com/ESA-APEx/apex_dispatch_api). In this notebook we will perform a small upscaling exercise for one of the services in the [APEx Algoritm Services Catalogue](https://algorithm-catalogue.apex.esa.int/), specfically the [Forest Fire Mapping](https://algorithm-catalogue.apex.esa.int/apps/parcel_delineation#execution-information). We will split up an area of interest in a 20x20km grid and execute this  through this upscaling task through the APEx Dispatch API.

In [36]:
import requests
import asyncio
import json
import websockets
from ipyleaflet import Map, GeoJSON

## Definition of parameters

In [39]:
dispatch_api = "localhost:8000"

In [14]:
application = "https://raw.githubusercontent.com/ESA-APEx/apex_algorithms/refs/heads/main/algorithm_catalog/vito/random_forest_firemapping/openeo_udp/random_forest_firemapping.json"
endpoint = "https://openeofed.dataspace.copernicus.eu"

In [7]:
area_of_interest =   {
        "coordinates": [
          [
            [
              -5.63017002925514,
              42.61315111519849
            ],
            [
              -7.147271480204438,
              42.61315111519849
            ],
            [
              -7.147271480204438,
              41.842023634350426
            ],
            [
              -5.63017002925514,
              41.842023634350426
            ],
            [
              -5.63017002925514,
              42.61315111519849
            ]
          ]
        ],
        "type": "Polygon"
      }
time_of_interest = ["2025-08-15", "2025-08-22"]

## Retrieval of the tiles
The first step in our upscaling exercise is to determine the different tiles to be processed based on the given `area_of_interest`. In this example we ask the dispatcher to split up the area in a `20x20km` grid. This results in a list of tiles that are visualised on the map.

In [8]:
tiles = requests.post(f"http://{dispatch_api}/tiles", json={
    "grid": "20x20km",
    "aoi": area_of_interest
}).json()

In [13]:
# Create a map centered at the approximate center of the area of interest
m = Map(center=[42.251628548555004, -6.37490034623255], zoom=8)
 
# Add the tiles (GeometryCollection) to the map
geo_json = GeoJSON(data=tiles)
m.add_layer(geo_json)

# Display the map
m

Map(center=[42.251628548555004, -6.37490034623255], controls=(ZoomControl(options=['position', 'zoom_in_text',…

## Launching the upscaling task

Next we trigger the upscaling task on the dispatcher. We provide the details of the processing jobs that need to be executed together with a `dimension`. This is an important parameter as this lets the dispatcher know how to scale up. In this case we are asking the dispatcher to scale up using the `spatial_extent`, creating a separate job for each geometry in the `values` section. The dispatcher will take care of all the rest. The result in the information on the created upscaling task.

In [20]:
upscaling_task = requests.post(f"http://{dispatch_api}/upscale_tasks", json={
    "title": "Forest Fire Detection",
    "label": "openeo",
    "service": {
        "endpoint": endpoint,
        "application": application
    },
    "parameters": {
        "temporal_extent": time_of_interest
    },
    "dimension": {
        "name": "spatial_extent",
        "values": tiles["geometries"]
    }
}).json()
upscaling_task

{'id': 4,
 'title': 'Forest Fire Detection',
 'label': 'openeo',
 'status': 'created'}

## Retrieve status of the upscaling task
We can now write a continuous monitoring process that fetches the status of the upscaling task and showcase the results on the map.

In [44]:
m = Map(center=[42.251628548555004, -6.37490034623255], zoom=8)
geo_json = GeoJSON(
    data={
        "type": "FeatureCollection",
        "features": []
    }
)
m.add_layer(geo_json)
display(m)

# Function to style jobs
def job_style(feature):
    color = {
        "created": "blue",
        "queued": "orange",
        "running": "yellow",
        "finished": "green",
        "canceled": "gray",
        "failed": "red"
    }.get(feature["properties"]["status"], "black")
    return {
        "color": color,
        "fillColor": color,
        "fillOpacity": 0.5
    }

async def listen_for_updates():
    ws_url = f"ws://{dispatch_api}/ws/upscale_tasks/{upscaling_task['id']}?interval=15"
    print(ws_url)
    async with websockets.connect(ws_url) as websocket:
        while True:
            message = await websocket.recv()
            status = json.loads(message)
            if status.get("jobs"):
                features = []
                for job in status["jobs"]:
                    features.append({
                        "type": "Feature",
                        "geometry": job["parameters"]["spatial_extent"],
                        "properties": {
                            "status": job["status"],
                        }
                    })
                geo_json.data = {
                    "type": "FeatureCollection",
                    "features": features
                }
                geo_json.style_callback = job_style
            if status.get("status") in ["finished", "canceled", "failed"]:
                break

# Run the websocket listener in the notebook
await listen_for_updates()

Map(center=[42.251628548555004, -6.37490034623255], controls=(ZoomControl(options=['position', 'zoom_in_text',…

ws://localhost:8000/ws/upscale_tasks/4


TimeoutError: timed out during opening handshake